# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import gkey

In [2]:
# Import File and Set DataFrame
cities_temps_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_temps_df

,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,3430443.0,Necochea,AR,-38.55,-58.74,11/09/20,79.00,40.0,100.0,8.01
1,2077963.0,Albany,AU,-35.02,117.89,11/09/20,48.20,93.0,75.0,23.04
2,2537881.0,Safi,MA,32.30,-9.24,11/09/20,66.29,80.0,0.0,19.28
3,3388847.0,São Félix do Xingu,BR,-6.64,-51.99,11/09/20,89.02,55.0,89.0,4.50
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
616,1282027.0,Malé,MV,4.17,73.51,11/09/20,82.40,83.0,75.0,5.82
617,2137773.0,Vao,NC,-22.67,167.48,11/09/20,70.09,73.0,55.0,17.40
618,1648082.0,Boyolangu,ID,-8.12,111.89,11/09/20,75.99,87.0,100.0,3.09
619,5983607.0,Inuvik,CA,68.35,-133.72,11/09/20,-3.12,83.0,100.0,11.41


In [3]:
# Drop Rows with NaN Values
cities_temps_df.dropna(inplace=True)
cities_temps_df.head()

,city_id,city_name,country,latitude,longitude,weather_date,temp,humidity,cloudiness,wind_speed
0,3430443.0,Necochea,AR,-38.55,-58.74,11/09/20,79.00,40.0,100.0,8.01
1,2077963.0,Albany,AU,-35.02,117.89,11/09/20,48.20,93.0,75.0,23.04
2,2537881.0,Safi,MA,32.30,-9.24,11/09/20,66.29,80.0,0.0,19.28
3,3388847.0,São Félix do Xingu,BR,-6.64,-51.99,11/09/20,89.02,55.0,89.0,4.50
5,6167817.0,Torbay,CA,47.67,-52.73,11/09/20,37.26,84.0,90.0,14.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure Gmaps
gmaps.configure(api_key=gkey)

# Use the Lat and Lng as locations
locations = cities_temps_df[["latitude","longitude"]]

#Add Humidity
humidity = cities_temps_df["humidity"]

In [6]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Add Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = cities_temps_df.loc[(cities_temps_df["wind_speed"] <= 10) & (cities_temps_df["cloudiness"] == 0) & \
                                   (cities_temps_df["temp"] >= 70) & (cities_temps_df["temp"] <= 80)].dropna()

narrowed_city_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Store Data into Variable hotel_df
hotel_df = pd.DataFrame(cities_temps_df, columns=["city_name", "country", "latitude", "longitude"])

hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Establish Params
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gkey}

In [ ]:
for index, row in hotel_df.iterrows():
    # Get city_name, latitude, longitude from DataFrame
    lat = row["latitude"]
    lng = row["longitude"]
    city_name = row["city_name"]
    
    # Add Keyword to params dict
    params["location"] = f"{lat},{lng}"

    # Assemble URL and Make API Request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract Results
    results = response['results']
    
    # Save Hotel Name to DataFrame
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # If no Hotel Available, Show Missing Field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 Second to Make Another API Request to Avoid SSL Error
    time.sleep(1)

# Print End of Search once Search is complete
print("-------End of Search-------")

In [ ]:
#Display Hotel Name in DataFrame
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""


# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [ ]:
# Plot Markers on Top of Heatmap
# Add Marker layer on Top of Heatmap
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig